# Setup 

In [ ]:
# Task specific
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

# The usuals
import pandas as pd
import numpy as np
import time

# Unique
from html import unescape

# OS etc
from os import listdir
from os.path import isfile, join

In [3]:
# Paths
PATH = '../data/'
OUTPUT = '../output_data/'
CREDS_PATH = '../collection/credentials/'

In [ ]:
# Filenames

creds_file = 'twitter_creds.json'

In [ ]:
# Get credentials

with open(CREDS_PATH + creds_file) as json_file:
    token_source = json.load(json_file)

consumer_key = token_source['consumer_key']
consumer_secret =token_source['consumer_secret']
access_token = token_source['access_token']
access_token_secret = token_source['access_token_secret']

In [ ]:
# Helper
def json_export(filename):
      
line = {'id': d + '_' + in_file.split('.')[0], 
      'faces_mode': mode_faces_scene, 
      'faces_max': max_faces_scene, 
      'frames_w_face': frames_with_face_scene}
with open ('./data/video_faces.jsonl', 'a') as f:
      json.dump(line, f)
      f.write('\n')   

# Main Twitter class

In [ ]:
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
    
        self.twitter_user = twitter_user
    
    def get_twitter_client_api(self):
        return self.twitter_client
    
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user, tweet_mode='extended').items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list
    
    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


class TwitterAuthenticator():
    
    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

class TwitterStreamer():
    
    def __init__(self):
        self.twitter_authenicator = TwitterAuthenticator()
        
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        
        listener = TwitterListener(fetched_tweets_filename)
        auth =  self.twitter_authenicator.authenticate_twitter_app()
        stream =  Stream(auth, listener)

        stream.filter(track=hash_tag_list)    

class TwitterListener(StreamListener):
    
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
    
    def on_data(self, data):
        try:
            print(data)
            with open (self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print('Error on data: %s' %str(e))
        return True
    
    def on_error(self, status):
        if status == 420:
            return False
            # rate limit
        print(status)
        
class TweetAnalyzer():
    def tweets_to_df(self, tweets):
        #tweets=unescape(tweet.full_text)

        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        return df


# Next

In [ ]:
# Activate API

twitter_cleint = TwitterClient()
tweet_analyzer = TweetAnalyzer()
api = twitter_cleint.get_twitter_client_api() 

In [ ]:

deleted_accounts = []

accounts = []
created_list = []
favourite = []
friends = []
followers = []
location = []
status_count = []

for user in usernames:
    try:
        account = api.get_user(screen_name=user) #count=10
        #print(account._json)
        accounts.append(account._json)
        
        print(account.created_at)
        
        created_list.append(account.created_at)
        followers.append(account.followers_count)
        friends.append(account.friends_count)
        favourite.append(account.favourites_count)
        status_count.append(account.statuses_count)
        
        time.sleep(1)
    
    except:
        deleted_accounts.append(user)
        pass
#tweets = api.user_timeline(screen_name='JPLehrke') #count=10
#print(dir(tweets[1]))
#test = api.user_timeline(screen_name='JPLehrke', count = 7, include_entities = True, tweet_mode='extended')
#print(account.created_at)
    
#print(tweets[6]._json)
#print(tweets[6].entities)

#df = tweet_analyzer.tweets_to_df(tweets)

In [ ]:
#usernames
deleted_accounts

In [ ]:
tweep = pd.DataFrame(list(zip(usernames, created_list, friends, followers, status_count)),
               columns =['user', 'created', 'friends', 'followers', 'status_count'])